# Accelerating deep neural networks with tensor decompositions

This notebook demonstrates how to use TensorLy for accelerating convolutional layers in PyTorch. It is based on this [blog post](https://jacobgil.github.io/deeplearning/tensor-decompositions-deep-learning).

We will show how to use Tucker and CP decompositions to find low rank approximations of convolutional layers.
Akin to prunning, this will allow use to reduce the number of parameters as well as make them faster. While the low-rank approximation can decrease the accuracy, this can be restored by training further.

Let's load a pretrained VGG-19 network, using torch vision:

In [1]:
from torchvision import models
model = models.vgg19(pretrained=True).eval()

The VGG-19 network has two parts:

- A convolutional part, stored in model.features. It's composed of 2D Convolutions, ReLU and max pooling layers.
   
   We are going to modify this part. 
 
 
- A fully connected part, stored in model.classifier. 
   
   It's composed of two large fully connected layers, and then a third fully connected layer that performs the final classification.

Lets look at one of the convolutional layers:

In [2]:
layer = model.features._modules['0']
print(layer)

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


This layer performs 64 different convolutions (the output channels), 

where each convolution has a 3x3 spatial size and is done on an image with 3 channels (the input channels).

If we call our convolutional kernel K, the input image X and the layer bias b, then the output from the layer, V will be:

$$ V(x, y, t) = \sum_i \sum_j \sum_sK(i, j, s, t)X(x-i, x-j, s) + b(t) $$ 


The weights of the layer are stored in a 4 dimensional tensor:


In [3]:
print(layer.weight.size())

torch.Size([64, 3, 3, 3])


We will use TensorLy to apply tensor decompositions on this 4 dimensional weight tensor.


## CP Decomposition on convolutional layers

This part is based on the paper titled [Speeding-up Convolutional Neural Networks Using Fine-tuned CP-Decomposition](https://arxiv.org/abs/1412.6553)

The idea is to approximate our 4 dimensional convolutional kernel with a CP decomposition of rank R:

$ K(i, j, s, t) = \sum_{r=1}^R K^x_r(i)K^y_r(j)K^s_r(s)K^t_r(t) $.

Plugging this into the formula for the convolutional layer output form above:

$ V(x, y, t) = \sum_r\sum_i \sum_j \sum_sK^x_r(i)K^y_r(i)K^s_r(s)K^t_r(t)X(x-i, y-j, s) $
$ = \sum_rK^t_r(t) \sum_i \sum_j K^x_r(i)K^y_r(j)\sum_sK^s_r(s)X(x-i, y-j, s) $ 

This gives us a recipe to do the convlution:

 1. First do a point wise (1x1xS) convolution with the kernek $K(r)$.
 
 This reduces the number of input channels from S to R.
 The convolutions will next be done on a smaller number of channels, making them faster.

 2. Perform seperable convolutions in the spatial dimensions with $K^x_r,K^y_r$. We can do this in PyTorch using grouped convolutions.

    **Like in [mobilenets](https://arxiv.org/abs/1704.04861) the convolutions are depthwise seperable, done in each channel separately.**
    **Unlike mobilenets the convolutions are also separable in the spatial dimensions.**

 3. Do another pointwise convolution to change the number of channels from R to T
 4. Finally, add the bias.
 
 
Notice the combination of pointwise and depthwise convolutions like in mobilenets. While with mobilenets you have to train a network from scratch to get this structure, here we can decompose an existing layer into this form.

As with mobile nets, to get the most speedup you will need a platform that has an efficient implementation of depthwise separable convolutions.

Now we can write a function that receives a PyTorch Conv2D layer, and creates the CP decompisition:

In [4]:
import tensorly as tl
from tensorly.decomposition import parafac
import torch
import torch.nn as nn

tl.set_backend('pytorch')

def cp_decomposition_conv_layer(layer, rank):
    """ Gets a conv layer and a target rank, 
        returns a nn.Sequential object with the decomposition
    """
    # Perform CP decomposition on the layer weight tensorly. 
    last, first, vertical, horizontal = \
        parafac(layer.weight.data, rank=rank, init='svd')

    pointwise_s_to_r_layer = torch.nn.Conv2d(in_channels=first.shape[0], \
            out_channels=first.shape[1], kernel_size=1, stride=1, padding=0, 
            dilation=layer.dilation, bias=False)

    depthwise_vertical_layer = torch.nn.Conv2d(in_channels=vertical.shape[1], 
            out_channels=vertical.shape[1], kernel_size=(vertical.shape[0], 1),
            stride=1, padding=(layer.padding[0], 0), dilation=layer.dilation,
            groups=vertical.shape[1], bias=False)

    depthwise_horizontal_layer = \
        torch.nn.Conv2d(in_channels=horizontal.shape[1],
            out_channels=horizontal.shape[1], 
            kernel_size=(1, horizontal.shape[0]), stride=layer.stride,
            padding=(0, layer.padding[0]), 
            dilation=layer.dilation, groups=horizontal.shape[1], bias=False)

    pointwise_r_to_t_layer = torch.nn.Conv2d(in_channels=last.shape[1], \
            out_channels=last.shape[0], kernel_size=1, stride=1,
            padding=0, dilation=layer.dilation, bias=True)

    pointwise_r_to_t_layer.bias.data = layer.bias.data

    depthwise_horizontal_layer.weight.data = \
        torch.transpose(horizontal, 1, 0).unsqueeze(1).unsqueeze(1)
    depthwise_vertical_layer.weight.data = \
        torch.transpose(vertical, 1, 0).unsqueeze(1).unsqueeze(-1)
    pointwise_s_to_r_layer.weight.data = \
        torch.transpose(first, 1, 0).unsqueeze(-1).unsqueeze(-1)
    pointwise_r_to_t_layer.weight.data = last.unsqueeze(-1).unsqueeze(-1)

    new_layers = [pointwise_s_to_r_layer, depthwise_vertical_layer, \
                    depthwise_horizontal_layer, pointwise_r_to_t_layer]
    
    return nn.Sequential(*new_layers)

Using numpy backend.
Using pytorch backend.


Let's check the result when applying this to our pretrained layer:

In [5]:
print('* Before the decomposition:')
print(layer)
layer_cp_decomposed = cp_decomposition_conv_layer(layer, rank=16)
print('\n* After the decomposition:')
print(layer_cp_decomposed)

* Before the decomposition:
Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

* After the decomposition:
Sequential(
  (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): Conv2d(16, 16, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), groups=16, bias=False)
  (2): Conv2d(16, 16, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), groups=16, bias=False)
  (3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
)


# Tucker decomposition on convolutional layers

The idea here is from the paper titled [*Compression of Deep Convolutional Neural Networks for Fast and Low Power Mobile Applications*](https://arxiv.org/abs/1511.06530).

Getting back to our kernel tensor K, we can approximate it using the Tucker decomposition:
$ K(i, j, s, t) = \sum_{r_1=1}^{R_1}\sum_{r_2=1}^{R_2}\sum_{r_3=1}^{R_3}\sum_{r_4=1}^{R_4}\sigma_{r_1 r_2 r_3 r_4} K^x_{r1}(i)K^y_{r2}(j)K^s_{r3}(s)K^t_{r4}(t) $

The Tucker decomposition has the useful property that it doesn't have to be decomposed along all the axis (modes).

Since the spatial dimensions are already quite small (3x3), it doesn't make a lot of sense to decompose those dimensions.

We can perform the decomposition along the input and output channels instead (a mode-2 decomposition):

$K(i, j, s, t) = \sum_{r_3=1}^{R_3}\sum_{r_4=1}^{R_4}\sigma_{i j r_3 r_4}(j)K^s_{r3}(s)K^t_{r4}(t)$

Like for CP decomposition, lets write the convolution formula and plug in the kernel decomposition: 
$ V(x, y, t) = \sum_i \sum_j \sum_s\sum_{r_3=1}^{R_3}\sum_{r_4=1}^{R_4}\sigma_{(i)(j) r_3 r_4}K^s_{r3}(s)K^t_{r4}(t)X(x-i, y-j, s) = \sum_i \sum_j \sum_{r_4=1}^{R_4}\sum_{r_3=1}^{R_3}K^t_{r4}(t)\sigma_{(i)(j) r_3 r_4} \sum_s\ K^s_{r3}(s)X(x-i, y-j, s) $ 

This gives us the following recipe for doing the convolution with Tucker Decomposition:

 1. Point wise convolution with $K^s_{r3}(s)$ for reducing the number of channels from S to $R_3$.

 2. Regular (not separable) convolution with $\sigma_{(i)(j) r_3 r_4} $.
 Instead of S input channels and T output channels like the original layer had,
 this convolution has $R_3$ input channels and $R_4$ output channels. If these ranks are smaller than S and T, this is were the speed gain comes from.

 3. Pointwise convolution with $K^t_{r4}(t)$ to get back to T output channels like the original convolution.
 4. Add the bias.

Now we can write a function that receives a PyTorch Conv2D layer, and creates the tucker decompisition:

In [6]:
from tensorly.decomposition import partial_tucker

def tucker_decomposition_conv_layer(layer, ranks):
    """ Gets a conv layer, 
        returns a nn.Sequential object with the Tucker decomposition.
    """
    core, [last, first] = \
        partial_tucker(layer.weight.data, \
            modes=[0, 1], ranks=ranks, init='svd')

    # A pointwise convolution that reduces the channels from S to R3
    first_layer = torch.nn.Conv2d(in_channels=first.shape[0], \
            out_channels=first.shape[1], kernel_size=1,
            stride=1, padding=0, dilation=layer.dilation, bias=False)

    # A regular 2D convolution layer with R3 input channels 
    # and R3 output channels
    core_layer = torch.nn.Conv2d(in_channels=core.shape[1], \
            out_channels=core.shape[0], kernel_size=layer.kernel_size,
            stride=layer.stride, padding=layer.padding, dilation=layer.dilation,
            bias=False)

    # A pointwise convolution that increases the channels from R4 to T
    last_layer = torch.nn.Conv2d(in_channels=last.shape[1], \
                                 out_channels=last.shape[0], kernel_size=1, stride=1,
                                 padding=0, dilation=layer.dilation, bias=True)

    last_layer.bias.data = layer.bias.data


    first_layer.weight.data = \
        torch.transpose(first, 1, 0).unsqueeze(-1).unsqueeze(-1)
    last_layer.weight.data = last.unsqueeze(-1).unsqueeze(-1)
    core_layer.weight.data = core


    new_layers = [first_layer, core_layer, last_layer]
    return nn.Sequential(*new_layers)

Applying this to our same pretrained layer:

In [7]:
print('* Before the decomposition:')
print(layer)
layer_tucker_decomposed = tucker_decomposition_conv_layer(layer, ranks=[16, 16])
print('\n* After the decomposition:')
print(layer_tucker_decomposed)

* Before the decomposition:
Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

* After the decomposition:
Sequential(
  (0): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
)


## Being careful with strides and dilations

Sometimes the convolutional layer will have a stride.
In this case, the last convolution among the decomposed parts, should have the original layer's stride, and the rest should have just a stride of one.
Since the last part of these decompositions is a 1x1 convolution, we can set the stride of the layer before it instead, to make it a bit faster. 

In case of dilations, all the spatial layers should preserve the dilation rate of the original layer.


## Accelerating the entire network
To accelerate the network, we can loop over the layers, and replace convolutional layers with their decomposition.
To show the decomposition is reducing the numebr of parameters, we can count the number of parameters in the network before and after the decomposition.


In [8]:
from PIL import Image
import requests
from io import BytesIO
from torchvision import transforms
from torch.autograd import Variable
import numpy as np

def count_params(network):
    return np.sum(np.prod(p.size()) for p in network.parameters())

# Apply the transformation to all the Convolutional layers
for index, module in enumerate(model.features._modules):
    if index > 0:
        layer = model.features._modules[module]
        if type(layer) is torch.nn.Conv2d:
            ranks = [layer.weight.size(0)//2, layer.weight.size(1)//2]
            model.features._modules[module] = tucker_decomposition_conv_layer(layer, ranks)

# Load another net without modification for comparison
original_model = models.vgg19(pretrained=True).eval()

params_before = count_params(original_model.features)
params_after = count_params(model.features)

print('Number of parameters before the decomposition: {params_before}'.format(params_before=params_before))
print('Number of parameters after the decomposition: {params_after}'.format(params_after=params_after))
print('Ratio: {ratio}'.format(ratio=float(params_after)/params_before))

Number of parameters before the decomposition: 20024384
Number of parameters after the decomposition: 7278656
Ratio: 0.3634896334389113


## Testing our compressed network

To show the network still is sensible after the decomposition, we will forward pass an image and check what is the highest scoring category before, and after the decomposition.

Let's take as an example this image:

![cat image](../images/cat.thumb)


First we define some helper functions to load the data etc:

In [9]:
import json

def image_from_url(url):
    """A function for loading an image from a URL, and preprocessing it 
    for the VGG19 network"""
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    
    resize = transforms.Resize((224, 224))
    normalize = transforms.NorJmalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    to_tensor = transforms.ToTensor()    
    
    return transforms.Compose([resize, transforms.ToTensor(), normalize])(img)

def image_from_disk(path):
    """A function for loading an image from a URL, and preprocessing it 
    for the VGG19 network"""
    img = Image.open(path)
    
    resize = transforms.Resize((224, 224))
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    to_tensor = transforms.ToTensor()    
    
    return transforms.Compose([resize, transforms.ToTensor(), normalize])(img)

def predict(net, img):
    """Passes an image through the net and returns the prediction"""
    input = Variable(img.unsqueeze(0), volatile=True)
    softmax = torch.nn.Softmax(1)
    output = softmax(net(input))
    return output.data.numpy()

# Use this if you want to test the model on an image from the web:
#url = "http://www.image-net.org/nodes/10/02123045/6c/6c34fe7c9d846c33a2f1a9b47a766b44ab4ec70d.thumb"
#img = image_from_url(url)



Now, let's apply both the original model and the model after decomposition to our cat image:

In [10]:
img = image_from_disk('../images/cat.thumb')

with open('../resources/imagenet_classes.json', 'r') as f:
    imagenet_classes = json.load(f)

output = predict(original_model, img)
prob, category = np.max(output), np.argmax(output)
category = imagenet_classes[str(category)]

output = predict(model, img)
prob, category = np.max(output), np.argmax(output)
category = imagenet_classes[str(category)]

print("Prediction BEFORE decomposition: '{category}' with score: {prob:.03}".format(category=category, prob=prob))
print("Prediction AFTER  decomposition: '{category}' with score: {prob:.03}".format(category=category, prob=prob))

Prediction BEFORE decomposition: 'tabby, tabby cat' with score: 0.419
Prediction AFTER  decomposition: 'tabby, tabby cat' with score: 0.419


![cat image](../images/cat.thumb)

Note that, although the class predicted is the same, the associated score is lower. To restore the original accuracy, we would have to retrain (fine-tune) the network for a few iterations on the original dataset.